In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
# load data
data = read_csv('final2.csv')
data.shape

/usr/local/Cellar/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(10356, 291)

In [2]:
x = data[list(range(288))]
y = data[[288]]
x.head()

,Heart_Rate,Heart_rate_Alarm_High,Heart_Rate_Alarm_Low,Arterial_Blood_Pressure_systolic,Arterial_Blood_Pressure_diastolic,Arterial_Blood_Pressure_mean,Arterial_Blood_Pressure_Alarm_Low,Arterial_Blood_Pressure_Alarm_High,Pulmonary_Artery_Pressure_systolic,Pulmonary_Artery_Pressure_diastolic,...,CO/CI_Change,Stroke_Volume_(SV_NICOM),Stroke_Volume_Index_(SVI_NICOM),SVI_Change,TFCd_(NICOM),TFCd0_(NICOM),Total_Peripheral_Resistance_(TPR)_(NICOM),Total_Peripheral_Resistance_Index_(TPRI)_(NICOM),Alsius_Bath_Temp,B_AGE
0,55.86,120.00,40.00,0.0,0.00,0.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,300
1,85.59,117.50,50.00,0.0,0.00,253.80,0,0,0,0,...,0,0,0,0,0,0,0,0,0,69
2,74.98,119.17,56.25,0.0,0.00,0.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,62
3,90.27,120.53,54.79,186.3,122.37,142.71,90,160,0,0,...,0,0,0,0,0,0,0,0,0,23
4,77.35,119.65,54.88,142.5,45.50,76.50,86,172,0,0,...,0,0,0,0,0,0,0,0,0,69


In [3]:
y.head()

,D_EXPIRE_FLAG
0,1
1,0
2,0
3,1
4,1


In [4]:
newy=[]
for item in range(len(y)):
    if(y.iloc[item,0]==0):
        newy.append(0)
    if(y.iloc[item,0]==1):
        newy.append(1)



In [5]:
#import the 
print len(newy)
from sklearn import preprocessing
name=np.array(x.columns)
xarray=np.array(x)

#范围0-1缩放标准化

min_max_scaler = preprocessing.MinMaxScaler()

X_scaler=min_max_scaler.fit_transform(x)

#查看特征的维度

print X_scaler.dtype

print x.shape
print len(newy)

10356
float64
(10356, 288)
10356


In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaler, newy, random_state=1)


print X_train.shape
print X_test.shape


(7767, 288)
(2589, 288)


from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB


# generate some data to play with





anova_filter = SelectKBest(chi2, k=5)
print anova_filter
#clf=SGDClassifier(loss="hinge", penalty="l2")
clf = MultinomialNB()
'''
classification algorithm: 
SVM,
SGD(Stochastic Gradient Descen),
KNN(Nearest Neighbors Classification)
MultinomialNB(multi naive bayes)
'''

anova_svm = Pipeline([('anova', anova_filter), ('svc', clf)])
# You can set the parameters using the names issued
# For instance, fit using a k of 10 in the SelectKBest
# and a parameter 'C' of the svm
anova_svm.fit(X_train, y_train)#可以使用‘_’符号直接链接某个属性
print anova_svm.named_steps  #实际上是一个字典
print type(anova_svm)
prediction = anova_svm.predict(X_test)
score = metrics.accuracy_score(y_test, prediction)
print prediction,type(prediction)
print score


# Test options and evaluation metric
num_folds = 10
num_instances = len(X_train)
seed = 7
scoring = 'accuracy'

# Spot Check Algorithms
from sklearn.tree import DecisionTreeRegressor



models = []
models.append(('DecisionTree', DecisionTreeRegressor()))
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('linearSVM', svm.LinearSVC()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    cv_results = cross_validation.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)

plt.savefig('1.png')
plt.show()

In [33]:
# Spot Check Algorithms
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import precision_score,recall_score
from sklearn import metrics


models = []
models.append(('DecisionTree', DecisionTreeRegressor()))
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('linearSVM', svm.LinearSVC()))

# evaluate each model in turn


recalls=[]

names = []
print "recall"
for name, model in models:
    clf=model.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    rec =recall_score(y_test, y_pred)
    f_pos, t_pos, thresh = metrics.roc_curve(y_test, y_pred)
    auc_area = metrics.auc(f_pos, t_pos)
    #plt.plot(f_pos, t_pos, lw=1, label='%s AUC = %.2f' % (name,auc_area))
    recalls.append(rec)
    names.append(name)
    msg = "%s: %f" % (name, rec)
    print(msg)




recall
DecisionTree: 0.496807
LR: 0.399745
LDA: 0.415070
KNN: 0.401022
CART: 0.490421
NB: 0.149425
linearSVM: 0.401022


In [48]:
print "precision"
precisions = []
names = []
for name, model in models:
    clf=model.fit(X_train,y_train)
    pred=clf.predict(X_test)
    precision =precision_score(y_test, pred)
    precisions.append(precision)
    names.append(name)
    msg = "%s: %f" % (name, precision)
    print(msg)

precision
DecisionTree: 0.518672
LR: 0.683406
LDA: 0.681342
KNN: 0.564748
CART: 0.529810
NB: 0.582090
linearSVM: 0.697778


In [45]:
from sklearn.metrics import f1_score
print "f1socre"
f1socres =[]
names = []
for name, model in models:
    clf=model.fit(X_train,y_train)
    pred=clf.predict(X_test)
    f1=f1_score(y_test, pred)
    f1socres.append(f1)
    names.append(name)
    msg = "%s: %f" % (name, f1)
    print(msg)

f1socre
DecisionTree: 0.497396
LR: 0.504432
LDA: 0.515873
KNN: 0.469007
CART: 0.502326
NB: 0.237805
linearSVM: 0.509327


In [64]:
def autolabel(rects): 
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x()+rect.get_width()/2., 1.03*height, "%s" % float(height))
itmes=[]
for item in f1socres:
    itmes.append(round(item,3))

fig = plt.figure()  
number=[0,1,2,3,4,5,6]
rect=plt.bar(number,itmes)  
plt.xticks(number, names, rotation=0)
plt.ylabel("f1socres")  
plt.title("f1socres bar chart")
plt.legend((rect,),(u"图例",))
autolabel(rect)
plt.savefig("f1socres.jpg")